In [1]:
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM, AutoModelForCausalLM
from transformers import LlamaConfig
from huggingface_hub import snapshot_download
from safetensors.torch import load_file, save_file, safe_open
from glob import glob
from tqdm import tqdm
import os
import torch

In [3]:
model = LlamaForCausalLM.from_pretrained(
    'mesolitica/malaysian-mistral-7b-32k-instructions-v5',
    config = LlamaConfig.from_pretrained('mesolitica/malaysian-mistral-7b-32k-instructions-v5'),
    use_flash_attention_2 = True,
    torch_dtype = torch.float16
)

You are using a model of type mistral to instantiate a model of type llama. This is not supported for all configurations of models and can yield errors.
The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
_ = model.cuda()

In [8]:
s = """
awak teloq
"""

messages = [
    {'role': 'user', 'content': s.strip()},
]
prompt = tokenizer.apply_chat_template(messages, tokenize = False)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=128,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] awak teloq [/INST]Maaf, saya tidak memahami perkataan "awak teloq" dalam bahasa Melayu. Bolehkah anda mengulangi atau memberikan maklumat lanjut?</s>
